[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OlivierGeorgeon/Developmental-AI-Lab/blob/master/docs/agent4.ipynb)

# THE AGENT WHO SHIFTED WITH THE CONTEXT

# Learning objectives

Upon completing this lab, you will be able to implement a developmental agent driven by interactional motivation that adapts its next action based on the context of the previously enacted interaction.

## Define the Interaction class

Let's define an Interaction class that will be useful to intitialize the agent and to memorize the context

In [1]:
class Interaction:
    """An interaction is a tuple (action, outcome) with a valence"""
    def __init__(self, action, outcome, valence):
        self.action = action
        self.outcome = outcome
        self.valence = valence

    def key(self):
        """ The key to find this interaction in the dictinary is the string '<action><outcome>'. """
        return f"{self.action}{self.outcome}"

    def __str__(self):
        """ Print interaction in the form '<action><outcome:<valence>' for debug."""
        return f"{self.action}{self.outcome}:{self.valence}"

    def __eq__(self, other):
        """ Interactions are equal if they have the same key """
        return self.key() == other.key()

## Define the Agent class

The agent is initialized with the list of interactions 

The previous action and the predicted outcome are memorized in the attribute `_intended_interaction`.

In [2]:
class Agent:
    """Creating our agent"""
    def __init__(self, _interactions):
        """ Initialize the dictionary of interactions"""
        self._interactions = {interaction.key(): interaction for interaction in _interactions}
        self._intended_interaction = self._interactions["00"]

    def action(self, _outcome):
        """ Tracing the previous cycle """
        previous_interaction = self._interactions[f"{self._intended_interaction.action}{_outcome}"]
        print(f"Action: {self._intended_interaction.action}, Prediction: {self._intended_interaction.outcome}, Outcome: {_outcome}, " 
              f"Prediction: {self._intended_interaction.outcome == _outcome}, Valence: {previous_interaction.valence})")

        """ Computing the next interaction to try to enact """
        # TODO: Implement the agent's decision mechanism
        intended_action = 0
        # TODO: Implement the agent's prediction mechanism
        intended_outcome = 0
        # Memorize the intended interaction
        self._intended_interaction = self._interactions[f"{intended_action}{intended_outcome}"]
        return intended_action

## Environment1 class

In [3]:
class Environment1:
    """ In Environment 1, action 0 yields outcome 0, action 1 yields outcome 1 """
    def outcome(self, _action):
        # return int(input("entre 0 1 ou 2"))
        if _action == 0:
            return 0
        else:
            return 1

## Environment2 class

In [4]:
class Environment2:
    """ In Environment 2, action 0 yields outcome 1, action 1 yields outcome 0 """
    def outcome(self, _action):
        if _action == 0:
            return 1
        else:
            return 0

## Environment3 class

Environment 3 yields outcome 1 only when the agent alternates actions 0 and 1

In [5]:
class Environment3:
    """ Environment 3 yields outcome 1 only when the agent alternates actions 0 and 1 """
    def __init__(self):
        """ Initializing Environment3 """
        self.previous_action = 0

    def outcome(self, _action):
        if _action == self.previous_action:
            _outcome = 0
        else:
            _outcome = 1
        self.previous_action = _action
        return _outcome

## Initialize the interactions 

In [6]:
interactions = [
    Interaction(0,0,-1),
    Interaction(0,1,1),
    Interaction(1,0,-1),
    Interaction(1,1,1),
    Interaction(2,0,-1),
    Interaction(2,1,1)
]

Interactions are initialized with their action, their outcome, and their valence:

|| outcome 0 | outcome 1|
|---|---|---|
| action 0| -1 | 1 |
| action 1 | -1 | 1 |
| action 2 | -1 | 1 |

## Instantiate the agent

In [7]:
a = Agent(interactions)

## Instantiate the environment 

In [8]:
e = Environment3()

## Test run the simulation

In [9]:
outcome = 0
for i in range(10):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)


Observe that in Envirnment3, the agent obtains only negative valences. To obtain a positive valence, it must select a different action on each interaction cycle. 

# PRELIMINARY EXERCISE

Execute the agent in Environment1. Observe that it obtains a negative valence. 

Execute the agent in Environment2. Observe that it obtains a positive valence. 

Now you see the goal of this assignement: design an agent that can obtain positive valences when it is run either in Environment1 or in Environment2 or in Environment3.

# ASSIGNMENT

Implement Agent4 that obtains positive valences in either Environment 1, 2, or 3. 

Agent4 must be able to predict the outcome resulting from its next action depending on the context of the previous interaction. 
Based on this prediction, it must select the action that will yield the heighest valence. 

To do so, at the end of cycle `t`, Agent4 must memorize `interaction_t = (action_t, outcome_t)` that was just enacted. 
The agent must choose the next `interaction_t+1` based on `interaction_t` (the context).
For each possible `action_t+1`, the agent must predict the expected `outcome_t+1`. 
Based on this prediction, it must select the action that yields the highest `valence_t+1`.

## Create Agent4 by overriding the class Agent

You may add any attribute and method you deem usefull to the class Agent4

In [10]:
class Agent4(Agent):
    def __init__(self, _interactions):
        """Creating our context-aware agent"""
        super().__init__(_interactions)
        # Context memory: stores sequences (previous_interaction, action) -> outcome
        self.context_memory = {}
        self.previous_interaction = None
        # Boredom management
        self.correct_predictions = 0
        self.boredom_threshold = 3
        
    def action(self, _outcome):
        """Tracing the previous cycle"""
        # Get the enacted interaction
        enacted_interaction = self._interactions[f"{self._intended_interaction.action}{_outcome}"]
        
        # Check if prediction was correct
        prediction_correct = (self._intended_interaction.outcome == _outcome)
        
        if prediction_correct:
            self.correct_predictions += 1
        else:
            self.correct_predictions = 0
        
        print(f"Action: {self._intended_interaction.action}, "
              f"Prediction: {self._intended_interaction.outcome}, "
              f"Outcome: {_outcome}, "
              f"Prediction_correct: {prediction_correct}, "
              f"Valence: {enacted_interaction.valence})")
        
        # Update context memory if we have a previous interaction
        if self.previous_interaction is not None:
            context_key = (self.previous_interaction.key(), self._intended_interaction.action)
            self.context_memory[context_key] = _outcome

        """Computing the next interaction to try to enact"""
        # TODO: Implement the agent's decision mechanism
        # ✅ IMPLÉMENTATION DU MÉCANISME DE DÉCISION CONTEXTUEL
        
        # If bored, add some randomness
        if self.correct_predictions >= self.boredom_threshold:
            import random
            if random.random() < 0.3:
                random_action = random.choice([0, 1, 2])
                context_key = (enacted_interaction.key(), random_action)
                if context_key in self.context_memory:
                    predicted_outcome = self.context_memory[context_key]
                else:
                    predicted_outcome = 0
                self._intended_interaction = self._interactions[f"{random_action}{predicted_outcome}"]
                self.previous_interaction = enacted_interaction
                self.correct_predictions = 0
                return random_action
        
        # Choose the best action based on context and valence
        best_interaction = None
        best_valence = -float('inf')
        
        for action in [0, 1, 2]:
            # TODO: Implement the agent's prediction mechanism
            # ✅ IMPLÉMENTATION DU MÉCANISME D'ANTICIPATION CONTEXTUEL
            
            # Predict outcome based on context
            context_key = (enacted_interaction.key(), action)
            
            if context_key in self.context_memory:
                predicted_outcome = self.context_memory[context_key]
            else:
                predicted_outcome = 0
            
            # Get the interaction for this action-outcome pair
            interaction_key = f"{action}{predicted_outcome}"
            candidate_interaction = self._interactions[interaction_key]
            
            # Evaluate this candidate
            if candidate_interaction.valence > best_valence:
                best_valence = candidate_interaction.valence
                best_interaction = candidate_interaction
            elif candidate_interaction.valence == best_valence and best_interaction is not None:
                import random
                if random.random() < 0.5:
                    best_interaction = candidate_interaction
        
        # Fallback if no good interaction found
        if best_interaction is None:
            best_interaction = self._interactions["00"]
        
        # Memorize for next cycle
        self.previous_interaction = enacted_interaction
        self._intended_interaction = best_interaction
        
        return best_interaction.action

## Test your Agent4 in Environment1

In [11]:
a = Agent4(interactions)
e = Environment1()
outcome = 0
for i in range(20):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1)
Action: 1, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 2, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1)
Action: 1, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 2, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 2, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 2, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1)
Action: 1, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Actio

## Test your Agent4 in Environment2

In [12]:
a = Agent4(interactions)
e = Environment2()
outcome = 0
for i in range(20):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1)
Action: 1, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 

## Test your Agent4 in Environment3

In [13]:
a = Agent4(interactions)
e = Environment3()
outcome = 0
for i in range(20):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1)
Action: 1, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 2, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 2, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1)
Action: 1, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 2, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 2, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 2, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 2, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action

## Test your Agent4 with interactions that have other valences

Replace the valences of interactions with your choice in the code below

In [14]:
# Choose different valence of interactions
interactions = [
    Interaction(0,0,1),
    Interaction(0,1,0),
    Interaction(1,0,-1),
    Interaction(1,1,1),
    Interaction(2,0,-1),
    Interaction(2,1,1)
]
# Run the agent
a = Agent4(interactions)
e = Environment3()
outcome = 0
for i in range(20):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 1)
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 0)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 0)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 1, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 1)
Action: 0,

## Test your agent in the Turtle environment 

In [15]:
# @title Install the turtle environment
!pip3 install ColabTurtle
from ColabTurtle.Turtle import *


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [16]:
# @title Initialize the turtle environment

BORDER_WIDTH = 20

class ColabTurtleEnvironment:

    def __init__(self):
        """ Creating the Turtle window """
        bgcolor("lightGray")
        penup()
        goto(window_width() / 2, window_height()/2)
        face(0)
        pendown()
        color("green")

    def outcome(self, action):
        """ Enacting an action and returning the outcome """
        _outcome = 0
        for i in range(10):
            # _outcome = 0
            if action == 0:
                # move forward
                forward(10)
            elif action == 1:
                # rotate left
                left(4)
                forward(2)
            elif action == 2:
                # rotate right
                right(4)
                forward(2)

            # Bump on screen edge and return outcome 1
            if xcor() < BORDER_WIDTH:
                goto(BORDER_WIDTH, ycor())
                _outcome = 1
            if xcor() > window_width() - BORDER_WIDTH:
                goto(window_width() - BORDER_WIDTH, ycor())
                _outcome = 1
            if ycor() < BORDER_WIDTH:
                goto(xcor(), BORDER_WIDTH)
                _outcome = 1
            if ycor() > window_height() - BORDER_WIDTH:
                goto(xcor(), window_height() -BORDER_WIDTH)
                _outcome = 1

            # Change color
            if _outcome == 0:
                color("green")
            else:
                # Finit l'interaction
                color("red")
                # if action == 0:
                #     break
                if action == 1:
                    for j in range(10):
                        left(4)
                elif action == 2:
                    for j in range(10):
                        right(4)
                break

        return _outcome

In [17]:
# @title Run the turtle environment
initializeTurtle()

# Parameterize the rendering
bgcolor("lightGray")
penup()
goto(window_width() / 2, window_height()/2)
face(0)
pendown()
color("green")
speed(10)

a = Agent4(interactions)
e = ColabTurtleEnvironment()

outcome = 0
for i in range(10):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 1)
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 0)
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: 0)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 0)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 0)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 0)
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: 0)


## Report 

Explain what you programmed and what results you observed. Export this document as PDF including your code, the traces you obtained, and your explanations below (no more than a few paragraphs):